# Zero Shot Text Classifier

## Development

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

### Load Pretrained Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('valhalla/distilbart-mnli-12-3')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('valhalla/distilbart-mnli-12-3')

### Preprocess Inputs

In [ ]:
text = """Last week I upgraded my iOS version and ever since then my phone has been overheating 
whenever I use your app.
"""

In [ ]:
classes = ['mobile', 'website', 'billing', 'account access']

In [ ]:
hypothesis_template = "This text is about {}"

In [ ]:
def preprocess(text, classes, hypothesis_template):
    '''Preprocesses a single input text to align with each class'''
    # create the hypotheses for each class
    hypotheses = [hypothesis_template.format(c) for c in classes]
    
    # preprocess the inputs
    inputs = tokenizer(
        [text] * len(classes), 
        hypotheses, 
        return_tensors='pt',
        truncation='only_first',
        padding=True
    )['input_ids']
    
    return inputs

In [ ]:
inputs = preprocess(text, classes, hypothesis_template)

### Make Predictions

In [ ]:
logits = model(inputs).logits

In [ ]:
print('Model Logits:', logits)

In [ ]:
def post_process(logits, classes):
    '''Post-processes the model output to get the entailment logits and get the class prediction'''
    idx = model.config.label2id['entailment']
    probabilities = torch.softmax(logits[:, idx], dim=0).tolist()
    
    output = []
    for i, prob in enumerate(probabilities):
        output.append(
            (classes[i], round(prob, 4))
        )
    
    return output

In [ ]:
post_process(logits, classes)

### Save Model and Tokenizer for Offline Inference

In [ ]:
tokenizer.save_pretrained('model_files')

In [ ]:
model.save_pretrained('model_files')

## Deployment

### Test Inference Script

In [ ]:
from inference import ZeroShotClassifier

In [ ]:
classifier = ZeroShotClassifier('model_files')

In [ ]:
text = """Tesla’s autonomous driving capability has inspired hair-raising antics on the road. 
Now the company is deploying an algorithm to determine whether customers have shown sufficiently 
sound judgement to use its “Full Self-Driving” software. What’s new: Starting this week, the beta-test 
version of Tesla’s latest self-driving update will be available only to drivers who have 
demonstrated safe driving. The beta program previously was open to about 2,000 drivers.
"""

In [ ]:
classes = ['technology', 'finance', 'sports', 'business']

In [ ]:
output = classifier.predict(text, classes)
print('Predictions', output)

### 1. Initialize Hub API Project
Open a terminal and run the following command,
```
hub init zero-shot
```

### 2. Integration

#### i. Copy the files from `model_files/` to `zero-shot/model/` folder in Hub API project

#### ii. Replace the `zero-shot/src/main.py` code with this,
```python
import json
import os
# Add your own import statements
from inference import ZeroShotClassifier

# This environment variable gives you the
# path to the directory of your model. You 
# can use this in your code to load model 
# and other large files
MODEL_DIR = os.getenv("MODEL_DIR")
classifier = ZeroShotClassifier(MODEL_DIR)

def handler(event, context):
    '''The main function which gets triggered on an API call for an AI model'''
    # ==================== DO NOT EDIT ====================
    if json.loads(event['inputs']).upper().strip == 'LOAD':
        return {
            'statusCode': 200,
            'body': json.dumps("PING RESPONSE")
        }
    # =====================================================
    # ++++++++ ADD YOUR INFERENCE CODE HERE ++++++++

    # Access your json encoded string input
    inputs = json.loads(event['inputs'])
    print('Inputs:', inputs)

    # Predict function of your model on the input
    output = classifier.predict(inputs['text'], inputs['classes'])
    print('Predictions:', output)
   
    return {
        'statusCode': 200,
        'body': json.dumps(output)
    }

```

#### iii. Add the libraries in `zero-shot/src/requirements.txt`
```
torch
transformers
```

### 3. Build and Deploy

Change directory into the `zero-shot` project folder in the terminal and then run the following commands,
```bash
hub build
hub deploy
```

### Test the Deployed API

In [ ]:
import os
import json
import requests
import base64

# This URL is a universal API Endpoint for HubAPI. 
HubAPI_URL = "https://api.cellstrathub.com/synchronous"

# Paste your API KEY here
API_KEY = "YOUR API KEY"

headers = {
  "x-api-key": API_KEY,
  "Content-Type": "application/json"
}

In [ ]:
%%time
payload = {
  "service_id": "zero-shot", # The name of the Hub API Project
  "input": json.dumps({
      'text': text,
      'classes': classes
  }) # the json-encoded input
}

# Send the POST request
response = requests.post(HubAPI_URL, headers=headers, data=json.dumps(payload)).json()

if response.get('statusCode') == 200:
    # Parse the output
    body = json.loads(response['body'])
    print('Predictions:', json.loads(body['output']))
else:
    print(response)